# Description
**Functionality**: This module formats the three way split, stratified variant-only Wikipedia Homograph Data (WHD) augmented with SWBD data with at least one example for each homograph pronunciation in each split. Formats data for XLNet token classification using Huggingface tools. 

**Use**: The  token classification model finetuning functionality from Huggingface expects CoNLL03-formatted data. The output from this module will be used to finetune models in order to predict pronunciation labels on homographs.

### Imports

In [1]:
import os
from glob import iglob
import pandas as pd
from tqdm import tqdm
from typing import List, Dict
import spacy
from transformers import AutoTokenizer

### Variables

In [2]:
#Paths 
BASE = "C:/Users/jseal/Dev/dissertation/Data/"
WHD_DATA_BASE = BASE + "WikipediaHomographData/data/"
METADATA = WHD_DATA_BASE + 'WikipediaHomographData.csv'
WHD_DATA_IN = WHD_DATA_BASE + "three_split_stratified_variant_data_corrected/"
WHD_DATA_OUT = BASE + "WHD_XLNet/variant_stratified_swbd_aug/"
LABELS = WHD_DATA_OUT + "labels.txt"

#Source paths
TRAIN = WHD_DATA_IN + "train_swbd_aug/"
DEV = WHD_DATA_IN + "dev_swbd_aug/"
TEST = WHD_DATA_IN + "test_swbd_aug/"
SOURCE_TSVS = [TRAIN, DEV, TEST]

#Destination paths
TRAIN_TSV = WHD_DATA_OUT + "train_tsvs/"
DEV_TSV = WHD_DATA_OUT + "dev_tsvs/"
TEST_TSV = WHD_DATA_OUT + "test_tsvs/"
DESTINATION_TSVS = [TRAIN_TSV, DEV_TSV, TEST_TSV]

SOURCE_DEST = zip(SOURCE_TSVS, DESTINATION_TSVS)

#Tmp Train, val, test splits in one file each
TRAIN_TMP = WHD_DATA_OUT + "train.txt.tmp"
DEV_TMP = WHD_DATA_OUT + "dev.txt.tmp"
TEST_TMP = WHD_DATA_OUT + "test.txt.tmp"
TMPS = [TRAIN_TMP, DEV_TMP, TEST_TMP]

TSVS_TMPS = zip(DESTINATION_TSVS, TMPS)

#Train, val, test splits in one file each
TRAIN_TXT = WHD_DATA_OUT + "train.txt"
DEV_TXT = WHD_DATA_OUT + "dev.txt"
TEST_TXT = WHD_DATA_OUT + "test.txt"
DESTINATIONS = [TRAIN_TXT, DEV_TXT, TEST_TXT]

TMPS_DESTS = zip(TMPS, DESTINATIONS)

ALL_PATHS_NEW = [WHD_DATA_OUT, TRAIN, DEV, TEST,
                 TRAIN_TSV, DEV_TSV, TEST_TSV]

ALL_FILES_NEW = [TRAIN_TMP, DEV_TMP, TEST_TMP, TRAIN_TXT, DEV_TXT, TEST_TXT, LABELS]

for path in ALL_PATHS_NEW: 
    if not os.path.exists(path): 
        os.mkdir(path)
for file in ALL_FILES_NEW : 
    with open(file, 'w') as f: 
        pass

#Tools
nlp = spacy.load('en_core_web_sm')

#Label variables
OUTSIDE = "O" #Label for all words that are not a homograph

#Options
pd.set_option('display.max_rows', None)

#Model info
MODEL_NAME = "xlnet-base-cased"


In [3]:
sent_nlp =  nlp("What : , - _ now ''.", disable=['parser', 'tagger', 'ner'])
for token in sent_nlp: 
    if token.is_alpha: 
        print(token)

What
now


### Functions

In [4]:
def get_tokens(sentence : str) -> List:
    sent_nlp =  nlp(sentence, disable=['parser', 'tagger', 'ner'])
    tokens = [token.text for token in sent_nlp if token.is_alpha]
    return tokens

def make_str(label : List) -> str: 
    return ' '.join(label)

def make_tsvs() -> None: 
    for PATHS in SOURCE_DEST: # Do this for train, test, valid
        for source in tqdm(iglob(PATHS[0] +'*.tsv')):
            print(PATHS[0])
            print(source)
            source_name = os.path.basename(source)
            df = pd.read_table(source)
            df = df[['homograph', 'wordid', 'sentence']]
            df['token'] = df.sentence.apply(lambda sentence : get_tokens(sentence))
            df = df.explode('token') # Get one row per token 
            for index, group in df.groupby(df.index):# Create one tsv per sentence; one line per token, label pair
                sentence_dicts = []
                for idx, row in group.iterrows():
                    sentence_dict = {}
                    token = row['token']
                    homograph = row['homograph']
                    sentence_dict['sent_id'] = "{}_{}".format(homograph, index)
                    sentence_dict['token'] = token
                    if token.lower() == homograph: # If the lowercase token is the same as the homograph, label with the wordid
                        sentence_dict['label'] = [row['wordid']]
                    else: 
                        sentence_dict['label'] = [OUTSIDE] # If the token is not the homograph, label with 'O' for 'outside'
                    sentence_dicts.append(sentence_dict)
                df = pd.DataFrame(sentence_dicts)
                df['label'] = df['label'].apply(make_str)
                new_f_name = PATHS[1] + source_name[:-4] + "_" + str(index) + '.txt'# Name file with homograph and sentence number
                df.to_csv(new_f_name, sep="\t", header=False, index=False) 
    
def make_tmps() -> None: 
    # Write temporary train, val, and test txt files from tsvs
    for tsv_dir, tmp in TSVS_TMPS:
        with open(tmp, 'w', encoding="utf8") as f_out: 
            for f in iglob(tsv_dir + "*"):
                with open(f, encoding="utf8") as example:
                    lines = example.readlines()
                    for line in lines: 
                        line_list = line.split('\t')
                        f_out.write(line_list[1] + '\t' + line_list[2])
                f_out.write('\n')    
                    
def make_txts() -> None:
    MAX_LENGTH = 128
    #Write train, val, and test txt files
    subword_len_counter = 0
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    MAX_LENGTH -= tokenizer.num_special_tokens_to_add()

    for tmp, outfile in tqdm(TMPS_DESTS):
        with open(tmp, "r", encoding="utf8") as f_p:
            with open(outfile, "w", encoding="utf8") as out_f: 
                for line in f_p:
                    line = line.rstrip()

                    if not line:
                        out_f.write(line +"\n")
                        subword_len_counter = 0
                        continue

                    token = line.split()[0]

                    current_subwords_len = len(tokenizer.tokenize(token))

                    # If token contains strange control characters like \x96 or \x95
                    # filter out the line
                    if current_subwords_len == 0:
                        continue

                    if (subword_len_counter + current_subwords_len) > MAX_LENGTH:
                        out_f.write("\n")
                        out_f.write(line +"\n")
                        subword_len_counter = current_subwords_len
                        continue

                    subword_len_counter += current_subwords_len

                    out_f.write(line + "\n")
                    
def make_labels() -> None: 
    sources = []
    for source_path in SOURCE_TSVS: 
        for source in iglob(source_path + "/*"):
            sources.append(pd.read_csv(source, sep='\t'))
    sources_df = pd.concat(sources)
    word_ids_unique = sources_df.wordid.unique()
    
    metadata_df = pd.read_csv(METADATA)
    wordids = metadata_df.wordid.tolist()
    with open(LABELS, 'w') as f:
        for wordid in wordids:
            if wordid in word_ids_unique:
                f.write("{}\n".format(wordid))
        f.write("{}\n".format('O'))

# Script

In [5]:
make_tsvs()
make_tmps()
make_txts()
make_labels()

0it [00:00, ?it/s]

C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/train_swbd_aug/
C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/train_swbd_aug\abuse.tsv


1it [00:00,  3.64it/s]

C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/train_swbd_aug/
C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/train_swbd_aug\abuses.tsv


2it [00:00,  3.57it/s]

...

126it [00:04, 24.51it/s]


C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/test_swbd_aug\upset.tsv
C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/test_swbd_aug/
C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/test_swbd_aug\use.tsv
C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/test_swbd_aug/
C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/test_swbd_aug\uses.tsv
C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/test_swbd_aug/
C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/test_swbd_aug\wind.tsv
C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratifi

128it [00:05, 25.46it/s]


C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/test_swbd_aug/
C:/Users/jseal/Dev/dissertation/Data/WikipediaHomographData/data/three_split_stratified_variant_data_corrected/test_swbd_aug\wound.tsv


3it [00:10,  3.50s/it]
